In [1]:
# Package Imports
import pandas as pd
import os

In [2]:
# Read data
books = pd.read_json('goodreads_books_young_adult.json', lines=True)

In [3]:
# View data
books

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[147734],US,,"[{'count': '1057', 'name': 'to-read'}, {'count...",B0056A00P4,true,4.04,B0056A00P4,...,,,,https://www.goodreads.com/book/show/12182387-t...,https://s.gr-assets.com/assets/nophoto/book/11...,12182387,4,285263,"The Passion (Dark Visions, #3)","The Passion (Dark Visions, #3)"
1,,2,[425995],US,,"[{'count': '1010', 'name': 'to-read'}, {'count...",B006KLYIAG,true,3.80,B006KLYIAG,...,,,,https://www.goodreads.com/book/show/20135365-h...,https://s.gr-assets.com/assets/nophoto/book/11...,20135365,5,18450480,Hope's Daughter,Hope's Daughter
2,0698143760,17,[493993],US,,"[{'count': '1799', 'name': 'fantasy'}, {'count...",,true,3.80,,...,3,,2014,https://www.goodreads.com/book/show/21401181-h...,https://images.gr-assets.com/books/1394747643m...,21401181,33,24802827,"Half Bad (Half Life, #1)","Half Bad (Half Life, #1)"
3,,9,[176160],US,eng,"[{'count': '7173', 'name': 'to-read'}, {'count...",B0042JSOQC,true,4.35,B004IYJDXY,...,,,,https://www.goodreads.com/book/show/10099492-t...,https://s.gr-assets.com/assets/nophoto/book/11...,10099492,152,10800440,Twelfth Grade Kills (The Chronicles of Vladimi...,Twelfth Grade Kills (The Chronicles of Vladimi...
4,0990662616,428,[],US,eng,"[{'count': '9481', 'name': 'to-read'}, {'count...",,false,3.71,B00MW0MTGE,...,10,Special Edition,2014,https://www.goodreads.com/book/show/22642971-t...,https://images.gr-assets.com/books/1406979059m...,22642971,1525,42144295,The Body Electric,The Body Electric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93393,,3,[294897],US,tur,"[{'count': '2641', 'name': 'to-read'}, {'count...",,false,4.21,,...,,,2013,https://www.goodreads.com/book/show/18221503-l...,https://images.gr-assets.com/books/1387020971m...,18221503,21,17831603,"Ölmem Gerekirse (Revenants, #3)","Ölmem Gerekirse (Revenants, #3)"
93394,,9,[238215],US,en-US,"[{'count': '12325', 'name': 'to-read'}, {'coun...",B003YFIVYA,true,3.79,B003YFIVYA,...,11,,2010,https://www.goodreads.com/book/show/8987191-th...,https://images.gr-assets.com/books/1385945375m...,8987191,42,7100578,"The Mockingbirds (The Mockingbirds, #1)","The Mockingbirds (The Mockingbirds, #1)"
93395,,55,[],US,,"[{'count': '1684', 'name': 'to-read'}, {'count...",,false,3.78,,...,7,,2007,https://www.goodreads.com/book/show/1885730.Jo...,https://images.gr-assets.com/books/1376736258m...,1885730,1094,1886909,Joel and Cat Set the Story Straight,Joel and Cat Set the Story Straight
93396,0385756224,14,[540527],US,eng,"[{'count': '2401', 'name': 'to-read'}, {'count...",,true,3.67,,...,8,,2015,https://www.goodreads.com/book/show/23636536-a...,https://images.gr-assets.com/books/1418831526m...,23636536,29,26112061,Another Day,Another Day


In [6]:
# List Columns
list(books.columns)

['isbn',
 'text_reviews_count',
 'series',
 'country_code',
 'language_code',
 'popular_shelves',
 'asin',
 'is_ebook',
 'average_rating',
 'kindle_asin',
 'similar_books',
 'description',
 'format',
 'link',
 'authors',
 'publisher',
 'num_pages',
 'publication_day',
 'isbn13',
 'publication_month',
 'edition_information',
 'publication_year',
 'url',
 'image_url',
 'book_id',
 'ratings_count',
 'work_id',
 'title',
 'title_without_series']

In [7]:
# Get rid of some columns (feel free to change)
books = books.drop(columns=['isbn',
                            'text_reviews_count',
                            'series',
                            'country_code',
                            'language_code',
                            'popular_shelves',
                            'asin',
                            'is_ebook',
                            'kindle_asin',
                            'format',
                            'link',
                            'publisher',
                            'publication_day',
                            'isbn13',
                            'publication_month',
                            'edition_information',
                            'publication_year',
                            'url',
                            'image_url'])

In [8]:
# View new data
books

,average_rating,similar_books,description,authors,num_pages,book_id,ratings_count,work_id,title,title_without_series
0,4.04,"[519546, 1295074, 21407416]",This is the final tale in the bestselling auth...,"[{'author_id': '50873', 'role': ''}, {'author_...",,12182387,4,285263,"The Passion (Dark Visions, #3)","The Passion (Dark Visions, #3)"
1,3.80,"[13400912, 13327517, 18107102, 15797097, 11472...",Life should be simple for Cassie.\nFor the sma...,"[{'author_id': '5395324', 'role': ''}]",,20135365,5,18450480,Hope's Daughter,Hope's Daughter
2,3.80,"[15728807, 17182499, 15673520, 16081758, 17842...",Wanted by no one.\nHunted by everyone.\nSixtee...,"[{'author_id': '7314532', 'role': ''}]",416,21401181,33,24802827,"Half Bad (Half Life, #1)","Half Bad (Half Life, #1)"
3,4.35,"[25861113, 7430195, 18765937, 6120544, 3247550...",It all comes down to this.\nVlad's running out...,"[{'author_id': '293603', 'role': ''}]",,10099492,152,10800440,Twelfth Grade Kills (The Chronicles of Vladimi...,Twelfth Grade Kills (The Chronicles of Vladimi...
4,3.71,"[20499652, 17934493, 13518102, 16210411, 17149...",The future world is at peace.\nElla Shepherd h...,"[{'author_id': '4018722', 'role': ''}]",351,22642971,1525,42144295,The Body Electric,The Body Electric
...,...,...,...,...,...,...,...,...,...,...
93393,4.21,"[11413298, 13508069, 7849034, 10552965, 155206...",Sevdigini Kurtarmak Icin Ne Kadarina Hazirsin?...,"[{'author_id': '3353516', 'role': ''}]",408,18221503,21,17831603,"Ölmem Gerekirse (Revenants, #3)","Ölmem Gerekirse (Revenants, #3)"
93394,3.79,"[8574517, 6440071, 6625948, 7036227, 6768407, ...",Some schools have honor codes.\nOthers have ha...,"[{'author_id': '3081700', 'role': ''}]",360,8987191,42,7100578,"The Mockingbirds (The Mockingbirds, #1)","The Mockingbirds (The Mockingbirds, #1)"
93395,3.78,"[1957880, 2490879, 8030321, 13152142, 6849661,...",,"[{'author_id': '348478', 'role': ''}, {'author...",240,1885730,1094,1886909,Joel and Cat Set the Story Straight,Joel and Cat Set the Story Straight
93396,3.67,"[26802671, 20662291, 23615716, 23870836, 21490...",The eagerly anticipated companion to David Lev...,"[{'author_id': '11664', 'role': ''}]",336,23636536,29,26112061,Another Day,Another Day
